<a href="https://colab.research.google.com/github/mori-assereto/DataAlchemy/blob/main/Final_Practical_Work_Part_2_STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Final Practical Work - Part 2: STS



# We install all necessary requirements

In [ ]:
!pip install datasets spacy fasttext transformers seqeval ipdb accelerate --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Step 1: Import libraries and pre-trained models

* We import the necessary libraries, such as pytorch and huggingface/transformers.
* We load our fine-tuned models for NLI and a reference model.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bi

# Step 2: Data preparation
* Use the Hugging Face dataset library to load the relevant datasets for STS in Spanish.
* We perform the necessary pre-processing on the data, such as tokenisation and encoding, to make it compatible with the pre-trained models.

## Data uploading

In [ ]:
#Download the semantic similarity dataset
from datasets import load_dataset
dataset = load_dataset("stsb_multi_mt", "es")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/es/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset["train"][0]

{'sentence1': 'Un avión está despegando.',
 'sentence2': 'Un avión está despegando.',
 'similarity_score': 5.0}

## Tokeniser

In [ ]:
# We print an example
tokenizer("Hello my name is John", "Goodbye my name is John", truncation=True, padding='longest')

NameError: ignored

In [ ]:
def tokenize_data(example, tokenizer):
    tokenizer.model_max_length = 128

    # Tokenise sentences
    tokenized = tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding='longest')
    # Returns a new dictionary with the tokens and the corresponding tag
    return {**tokenized, "labels": example["similarity_score"]}

# Transforms and formats datasets

# We apply preprocessing to each example dataset for each model.
dataset = dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True, batch_size=1000)

# Converting data to PyTorch tensor lists
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','labels'])

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

## Dataloader

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def create_dataloaders(dataset, batch_size):
    # Set the collate_fn
    def collate_fn(batch):
        # input_ids the elements of the batch
        input_ids_batch = [example['input_ids'] for example in batch]
        token_type_ids_batch = [example['token_type_ids'] for example in batch]
        attention_mask_batch = [example['attention_mask'] for example in batch]
        labels_batch = [example['labels'] for example in batch]

        # Applying filler to the tensioners
        input_ids_batch = pad_sequence(input_ids_batch, batch_first=True, padding_value=0)
        token_type_ids_batch = pad_sequence(token_type_ids_batch, batch_first=True, padding_value=0)
        attention_mask_batch = pad_sequence(attention_mask_batch, batch_first=True, padding_value=0)

        # Convert labels to tensors
        labels_batch = torch.tensor(labels_batch)

        # Return batch as a dictionary
        return {
            'input_ids': input_ids_batch.to(device),
            'token_type_ids': token_type_ids_batch.to(device),
            'attention_mask': attention_mask_batch.to(device),
            'labels': labels_batch.to(device)
        }


    # Create DataLoaders
    train_loader = DataLoader(dataset['train'], batch_size=batch_size,
                              shuffle=True, collate_fn=collate_fn)

    val_loader = DataLoader(dataset['dev'], batch_size=batch_size,
                            collate_fn=collate_fn)

    test_loader = DataLoader(dataset['test'],batch_size=batch_size,
                             collate_fn=collate_fn)

    return train_loader, val_loader, test_loader


# Create the DataLoaders for each model and dataset
batch_size = 50
train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)


# Step 3: Fine-tuning for STS and Training

* Define a regression model that can process the STS scores.
* Define a suitable loss function for the STS task
* We use an optimiser, such as Adam, to update the parameters of the network.
* We calculate the loss and perform backpropagation to update the network weights.


In [ ]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self, base_model):
        super(RegressionModel, self).__init__()
        self.base_model = base_model
        self.regression = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids, attention_mask=attention_mask).pooler_output
        regression_output = self.regression(outputs)
        return regression_output.squeeze()

## Training function

In [ ]:
from scipy.stats import pearsonr, spearmanr

# We define the evaluation metric
def pearson(predictions, labels):
  pearson_corr, _ = pearsonr(predictions, labels)
  return pearson_corr

def spearman(predictions, labels):
  spearman_corr, _ = spearmanr(predictions, labels)
  return spearman_corr

In [ ]:
import numpy as np
from tqdm.auto import tqdm
from accelerate import Accelerator
from transformers import AdamW, get_linear_schedule_with_warmup

def fine_tune_model(train_loader, val_loader, model, num_epochs):
  accelerator = Accelerator()
  device = accelerator.device
  model = model.to(device)

  loss_fn = nn.MSELoss()

  optimizer = AdamW(model.parameters(), lr=1e-5)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                              num_training_steps=len(train_loader) * num_epochs)

  # Start the training loop
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0.0

      for batch in tqdm(train_loader):
          input_ids = batch['input_ids']
          attention_mask = batch['attention_mask']
          token_type_ids = batch['token_type_ids']
          labels = batch['labels']

          optimizer.zero_grad()

          # Forward pass
          predictions = model(input_ids, attention_mask)

          # Compute the loss
          loss = loss_fn(predictions, labels)

          # Backpropagate the loss
          accelerator.backward(loss)
          optimizer.step()
          scheduler.step()

          total_loss += loss.item() * len(labels)

      train_pearson = pearson(predictions.cpu().detach().numpy(), labels.cpu().detach().numpy())
      train_spearman = spearman(predictions.cpu().detach().numpy(), labels.cpu().detach().numpy())

      # Evaluate the model on the test set
      model.eval()
      val_loss= 0.0
      val_predictions = []
      val_labels = []

      with torch.no_grad():
          for batch in val_loader:
              input_ids = batch['input_ids']
              attention_mask = batch['attention_mask']
              token_type_ids = batch['token_type_ids']
              labels = batch['labels']

              # Forward pass
              predictions = model(input_ids, attention_mask)

              # Compute the loss
              loss = loss_fn(predictions, labels)
              val_loss += loss.item()

              val_predictions.extend(predictions.tolist())
              val_labels.extend(labels.tolist())

          # Calculate the Pearson correlation
          val_pearson = pearson(val_predictions, val_labels)
          val_spearman = spearman(val_predictions, val_labels)

          # Imprimir o guardar las métricas de entrenamiento y validación
          print(f"Epoch {epoch+1}/{num_epochs}:")
          print(f"  Train Loss: {total_loss:.4f} | Train Pearson: {train_pearson:.4f} | Train Spearman: {train_spearman:.4f}")
          print(f"  Val Loss: {val_loss:.4f} | Val Pearson: {val_pearson:.4f} | Val Spearman: {val_spearman:.4f}")

# BERT Training

In [ ]:
# Training using the BERT model
num_epochs = 1
# Creates training task for the model
model_bert = RegressionModel(model)
fine_tune_model(train_loader, val_loader, model_bert, num_epochs)

  0%|          | 0/115 [00:00<?, ?it/s]

Epoch 1/1:
  Train Loss: 9681.7479 | Train Pearson: 0.8149 | Train Spearman: 0.8287
  Val Loss: 32.0934 | Val Pearson: 0.7435 | Val Spearman: 0.7712


### Tabular validation of results
* We create test set predictions - so far not used.
* We generate tables to visualise the quality of the predictions.

In [ ]:
def predict_test(dataset_test, test_loader, model):
  test_predictions = []
  test_labels = []
  with torch.no_grad():
    for batch in tqdm(test_loader):
      input_ids = batch['input_ids']
      attention_mask = batch['attention_mask']
      token_type_ids = batch['token_type_ids']
      labels = batch['labels']

      outputs = model(input_ids, attention_mask)
      test_predictions.extend(outputs.tolist())

  results = dataset_test.select_columns(["sentence1","sentence2","labels"])
  results = results.add_column("predictions", test_predictions)
  return results

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-10)
    df = pd.DataFrame()
    df["sentence1"] = dataset['sentence1'][pick:pick+10]
    df["sentence2"] = dataset['sentence2'][pick:pick+10]
    df["labels"] = dataset['labels'][pick:pick+10]
    df["predictions"] = dataset['predictions'][pick:pick+10]
    display(HTML(df.to_html()))

In [ ]:
results = predict_test(dataset['test'], test_loader, model_bert)
show_random_elements(results)

  0%|          | 0/28 [00:00<?, ?it/s]

,sentence1,sentence2,labels,predictions
0,"Pero en los primeros 30 segundos después de que Young entrara en el ring, la familia supo que era un partido desigual, dijo Meyers.","Pero en los primeros 30 segundos del combate, los miembros de la familia sabían que era un combate desigual, dijo Jodie Meyers, la hermana de Stacy Young.",4.200,3.775560
1,"Singapur no informó el miércoles de ningún caso sospechoso de SARS, pero las autoridades pusieron en cuarentena a 70 personas que tuvieron contacto con el paciente taiwanés.","Aún así, Singapur puso en cuarentena a 70 personas que habían estado en estrecho contacto con el científico.",3.000,2.879720
2,"Aguas abajo en Mount Vernon, se esperaba que el río Skagit llegara a su cima a 36 pies, 8 pies por encima de la etapa de inundación, esta noche, dijo Burke.","Se esperaba que el Skagit llegara a la cima durante la noche a 38 pies en Mount Vernon, 10 pies por encima de la etapa de inundación, dijo el Servicio Meteorológico Nacional.",3.000,3.773041
3,"El lunes, el Dow bajó 5.25, o 0.1 por ciento, a 8,983.80, habiendo perdido 2.3 por ciento la semana pasada.","Al principio del comercio, el promedio industrial del Dow Jones bajó 39.94, o 0.4 por ciento, a 8,945.50, habiendo caído 3.61 puntos el lunes.",1.600,2.624242
4,"Los heridos fueron llevados a los hospitales de Jefferson City y Columbia, una ciudad universitaria a unas 30 millas al norte.","Los heridos fueron llevados a los hospitales de Jefferson City y Columbia, a unos 48 km al norte; sus condiciones no estaban disponibles de inmediato.",4.000,3.582728
5,"El dólar estaba a 117,85 yenes frente a la moneda japonesa, con una subida del 0,1 por ciento.","Frente al franco suizo, el dólar se situó en 1.3289 francos, con un aumento del 0,5% en el día.",1.333,2.403599
6,"Además de O'Connor, a la opinión mayoritaria de Rehnquist se sumaron los jueces David Souter, Ruth Bader Ginsburg y Stephen Breyer.","Junto a él, en la opinión mayoritaria, estaban los jueces Sandra Day O'Connor, David H. Souter, Ruth Bader Ginsburg y Stephen G. Breyer.",3.600,3.608065
7,"Martin, de 58 años, será liberado hoy después de cumplir dos tercios de su condena de cinco años por el homicidio de Fred Barras, de 16 años.",Martin cumplió dos tercios de una sentencia de cinco años por el homicidio de Barras y por herir a Fearon.,3.000,3.484483
8,"Los accionistas de IDEC poseerían el 50,5 por ciento de las acciones de la compañía combinada, y los accionistas de Biogen el 49,5 por ciento restante.","Las empresas dijeron que los poseedores de Idec serían dueños de alrededor del 50,5 por ciento de las acciones de la empresa combinada.",3.000,3.648870
9,"Sin embargo, el gurú del código abierto insistió: ""Este ataque estuvo mal, y fue peligroso para nuestros objetivos"".","""Este ataque estuvo mal, y fue peligroso para nuestros objetivos"", dijo Raymond.",3.500,3.549728


# BERT fine-tuned training
* We repeat the procedure for our model

## Data prep

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("themagus/RAG_S_BERT")
model = AutoModel.from_pretrained("themagus/RAG_S_BERT")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize
dataset = dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True, batch_size=1000)
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','labels'])

# Dataloader
batch_size = 50
train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

## Train

In [ ]:
# Training with the BERT fine-tuned model for NLI
num_epochs = 1
# Create training task for the model
model = RegressionModel(model)
fine_tune_model(train_loader, val_loader, model, num_epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/115 [00:00<?, ?it/s]

Epoch 1/1:
  Train Loss: 12061.4040 | Train Pearson: 0.7593 | Train Spearman: 0.7286
  Val Loss: 44.1435 | Val Pearson: 0.6869 | Val Spearman: 0.6983


## Results

In [ ]:
results = predict_test(dataset['test'], test_loader, model)
show_random_elements(results)

  0%|          | 0/28 [00:00<?, ?it/s]

,sentence1,sentence2,labels,predictions
0,La chica tiene algo en la cabeza.,La mujer tiene algo con ella.,2.6,2.968741
1,Gente que hace una estructura con las cartas.,Dos personas están juntas y una está usando una pieza de ropa de una manera diferente a la prevista.,0.2,1.940574
2,Hay cuatro personas afuera.,Cinco personas están afuera.,3.0,2.873240
3,Las chicas se toman de la mano con ropa azul,Los jóvenes llevan vaqueros con sus camisas rojas y verdes.,1.8,1.826441
4,Un hombre está demostrando cómo trabajar con la arcilla mientras al menos dos personas observan.,Al menos una de las tres mujeres está hablando con las otras.,0.6,2.132540
5,Un perro tiene un suéter.,Un hombre con chaqueta de traje está sentado en un sofá rojo con un gato.,0.6,1.443534
6,Escultor aplicando hormigón a una gran estatua de figura humana de pie sin cabeza.,Una persona está de pie frente a un edificio sosteniendo en lo alto unos contrachapados,0.2,1.843510
7,Un chico de pelo oscuro está en el agua y está mirando hacia arriba.,El hombre tiene el pelo corto.,0.6,1.869149
8,Un poste está detrás de una persona que está jugando al escondite con un niño.,Hay un niño con una pelota,0.2,1.742457
9,Una mujer está a punto de pintar.,Un chico está a punto de tomar una foto.,1.0,1.779450


# RoBERTa fine-tuned training
* We repeat the procedure for our model

## Data prep

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("themagus/RAG_S_ROBERTA")
model = AutoModel.from_pretrained("themagus/RAG_S_ROBERTA")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize
dataset = dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True, batch_size=1000)
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','labels'])

# Dataloader
batch_size = 50
train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

## Train

In [ ]:
# Training with the RoBERTa fine-tuned model for NLI
num_epochs = 1
# Create training task for the model
model = RegressionModel(model)
fine_tune_model(train_loader, val_loader, model, num_epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/115 [00:00<?, ?it/s]

Epoch 1/1:
  Train Loss: 8654.1465 | Train Pearson: 0.7715 | Train Spearman: 0.7355
  Val Loss: 21.5679 | Val Pearson: 0.8275 | Val Spearman: 0.8369


## Results

In [ ]:
results = predict_test(dataset['test'], test_loader, model)
show_random_elements(results)

  0%|          | 0/28 [00:00<?, ?it/s]

,sentence1,sentence2,labels,predictions
0,Un terrorista suicida mata a 12 personas en una iglesia de Nigeria,Un terrorista suicida mata a 21 personas en el noroeste de Pakistán,0.75,1.150408
1,EE.UU. dice estar dispuesto a ayudar al activista chino ciego,EE.UU. está listo para ayudar al activista chino: oficial,3.40,3.820354
2,Los habitantes de Gaza disparan 2 cohetes contra Israel; no hay heridos,2 cohetes de Gaza impactaron en Israel; no hubo heridos,4.80,3.727401
3,Berri dice que está listo para ayudar a lanzar el gobierno de unidad nacional,La percepción de España ha cambiado dramáticamente con este gobierno,0.40,1.114545
4,13 muertos en los ataques con bombas en el Iraq,16 muertos en los ataques al norte del Iraq,1.80,2.408363
5,El huracán Isaac se mueve tierra adentro después de tocar tierra,El huracán Isaac golpea la costa de EE.UU. cerca de Nueva Orleans,2.40,2.209808
6,Nigeria llora la muerte de más de 193 personas en un accidente aéreo,Nigeria abre una investigación sobre el mortal accidente aéreo,1.60,2.450743
7,La CPI encarcela a un señor de la guerra congoleño durante 14 años por culpa de los niños soldados,Señor de la guerra congoleño condenado a 14 años en un caso histórico - video,3.20,3.148677
8,"7 soldados muertos, 56 heridos en el ataque del PKK en el este de Turquía","3 muertos, 14 heridos en ataques en el Iraq",0.40,0.782935
9,El aeropuerto de Schiphol fue parcialmente evacuado después de la amenaza de bomba,El aeropuerto de Schiphol fue evacuado debido a la amenaza de bomba,4.00,3.902488


In [ ]:
import fasttext
import fasttext.util

fasttext.util.download_model('es', if_exists='ignore')  # Spanish
model_fasttext = fasttext.load_model('cc.es.300.bin')

In [ ]:
import numpy as np

def get_sentence_vectors_batch(batch, model_fasttext):
    sentence1_vectors = [model_fasttext.get_sentence_vector(sentence) for sentence in batch["sentence1"]]
    sentence2_vectors = [model_fasttext.get_sentence_vector(sentence) for sentence in batch["sentence2"]]
    return {"sentence1_vector": sentence1_vectors, "sentence2_vector": sentence2_vectors}

batch_size = 1000
train_dataset = dataset["train"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)
val_dataset = dataset["dev"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)
test_dataset = dataset["test"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarities_batch(batch):
    sentence1_vectors = np.array(batch["sentence1_vector"])
    sentence2_vectors = np.array(batch["sentence2_vector"])
    similarities = cosine_similarity(sentence1_vectors, sentence2_vectors).diagonal()
    return {"similarities": similarities.tolist()}

train_dataset = train_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)
val_dataset = val_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)
test_dataset = test_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

# Training RoBERTa

## Model initialisation

In [ ]:
# Model and tokeniser initialisation
model_name_roberta = "PlanTL-GOB-ES/roberta-base-bne"

# We import the tokenisers corresponding to the models.
tokenizer_roberta = AutoTokenizer.from_pretrained(model_name_roberta)

# We import pre-trained models
model_roberta = AutoModel.from_pretrained(model_name_roberta)

# Configure the running devices (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# We move the models to the GPU, if available.
model_roberta = model_roberta.to(device)

Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream tas

## Training

In [ ]:
# Training with the RoBERTa model
num_epochs = 1
# Create training task for the model
model_roberta = RegressionModel(model_roberta)
fine_tune_model(train_loader, val_loader, model_roberta, num_epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/115 [00:00<?, ?it/s]

Epoch 1/1:
  Train Loss: 12851.4083 | Train Pearson: 0.7326 | Train Spearman: 0.6841
  Val Loss: 40.5454 | Val Pearson: 0.6611 | Val Spearman: 0.6585


### Tabular validation of results


In [ ]:
results = predict_test(dataset['test'], test_loader, model_roberta)
show_random_elements(results)

  0%|          | 0/28 [00:00<?, ?it/s]

,sentence1,sentence2,labels,predictions
0,Tal vez esto podría funcionar para ti.,Tal vez esto pueda funcionar para ti.,5.0,3.473458
1,"Depende de lo que quieras hacer a continuación, y de dónde quieras hacerlo.","Pueden, pero la forma de hacerlo depende de lo que tengas disponible.",1.0,2.715275
2,El microondas sería tu mejor apuesta.,Tu mejor apuesta es la investigación.,0.0,2.130100
3,"En los EE.UU., dependerá de la escuela.",Esto dependerá de la universidad.,4.0,2.753767
4,No es una buena idea.,"No es sólo una buena idea, es una excelente idea.",1.0,3.401118
5,Hay algunas cosas que creo que deberías hacer.,Hay algunas cosas mínimamente efectivas que puedes hacer a nivel personal.,4.0,2.718715
6,He tenido este mismo problema.,Yo tuve esta misma situación.,3.0,1.563561
7,Esto no es una buena idea.,No es una buena idea.,5.0,3.260192
8,"Me di cuenta de que ya hay una respuesta aceptada, pero creo que añadiría mis dos centavos.","Sé que es una vieja pregunta, pero creo que debería añadir mis dos céntimos.",5.0,3.072496
9,"Sí, hay una razón para no hacer lo que te propones.","Sí, hay una diferencia química.",0.0,3.083890


# Fasttext

## Model initialisation

In [ ]:
import fasttext
import fasttext.util

fasttext.util.download_model('es', if_exists='ignore')  # Spanish
model_fasttext = fasttext.load_model('cc.es.300.bin')

## Data Preparation

In [ ]:
import numpy as np

def get_sentence_vectors_batch(batch, model_fasttext):
    sentence1_vectors = [model_fasttext.get_sentence_vector(sentence) for sentence in batch["sentence1"]]
    sentence2_vectors = [model_fasttext.get_sentence_vector(sentence) for sentence in batch["sentence2"]]
    return {"sentence1_vector": sentence1_vectors, "sentence2_vector": sentence2_vectors}

batch_size = 1000
train_dataset = dataset["train"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)
val_dataset = dataset["dev"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)
test_dataset = dataset["test"].map(lambda batch: get_sentence_vectors_batch(batch, model_fasttext), batched=True, batch_size=batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

### Similarity calculation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarities_batch(batch):
    sentence1_vectors = np.array(batch["sentence1_vector"])
    sentence2_vectors = np.array(batch["sentence2_vector"])
    similarities = cosine_similarity(sentence1_vectors, sentence2_vectors).diagonal()
    return {"similarities": similarities.tolist()}

train_dataset = train_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)
val_dataset = val_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)
test_dataset = test_dataset.map(calculate_similarities_batch, batched=True, batch_size=batch_size)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
train_scores = dataset['train']['similarity_score']
val_scores = dataset['dev']['similarity_score']
test_scores = dataset['test']['similarity_score']

train_similarities = [item['similarities'].tolist() for item in train_dataset]
val_similarities = [item['similarities'].tolist() for item in val_dataset]
test_similarities = [item['similarities'].tolist() for item in test_dataset]

## Evaluation Model

In [ ]:
mean_train_similarity = np.mean(train_similarities)
mean_val_similarity = np.mean(val_similarities)
mean_test_similarity = np.mean(test_similarities)

print(f"Promedio de similaridad en el conjunto de entrenamiento: {mean_train_similarity}")
print(f"Promedio de similaridad en el conjunto de validación: {mean_val_similarity}")
print(f"Promedio de similaridad en el conjunto de prueba: {mean_test_similarity}")

Promedio de similaridad en el conjunto de entrenamiento: 0.8283866537117008
Promedio de similaridad en el conjunto de validación: 0.8053415736556053
Promedio de similaridad en el conjunto de prueba: 0.8276980085767119


In [ ]:
from scipy.stats import pearsonr

train_corr, _ = pearsonr(train_similarities, train_scores)
val_corr, _ = pearsonr(val_similarities, val_scores)
test_corr, _ = pearsonr(test_similarities, test_scores)

print(f"Correlación de Pearson en el conjunto de entrenamiento: {train_corr}")
print(f"Correlación de Pearson en el conjunto de validación: {val_corr}")
print(f"Correlación de Pearson en el conjunto de prueba: {test_corr}")


Correlación de Pearson en el conjunto de entrenamiento: 0.5215977216628838
Correlación de Pearson en el conjunto de validación: 0.5794524260200727
Correlación de Pearson en el conjunto de prueba: 0.4804286547635741


In [ ]:
from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(train_scores, train_similarities)
val_mse = mean_squared_error(val_scores, val_similarities)
test_mse = mean_squared_error(test_scores, test_similarities)

print(f"MSE en el conjunto de entrenamiento: {train_mse}")
print(f"MSE en el conjunto de validación: {val_mse}")
print(f"MSE en el conjunto de prueba: {test_mse}")

MSE en el conjunto de entrenamiento: 5.492375385265878
MSE en el conjunto de validación: 4.497385289590478
MSE en el conjunto de prueba: 5.346998377542148


In [ ]:
from scipy.stats import spearmanr

train_spearman, _ = spearmanr(train_similarities, train_scores)
val_spearman, _ = spearmanr(val_similarities, val_scores)
test_spearman, _ = spearmanr(test_similarities, test_scores)

print(f"Correlación de Spearman en el conjunto de entrenamiento: {train_spearman}")
print(f"Correlación de Spearman en el conjunto de validación: {val_spearman}")
print(f"Correlación de Spearman en el conjunto de prueba: {test_spearman}")

Correlación de Spearman en el conjunto de entrenamiento: 0.5179303974163642
Correlación de Spearman en el conjunto de validación: 0.6042232319275204
Correlación de Spearman en el conjunto de prueba: 0.5047885024099179
